In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
!git clone -b wreckin https://github.com/Remi-Lejeune/deep-learning-project/

Cloning into 'deep-learning-project'...
remote: Enumerating objects: 626, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (66/66), done.
^C


In [3]:
%cd deep-learning-project/

/content/deep-learning-project


In [4]:
#install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#check python version
!python --version
#3.9.16

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,691 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [808 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,135 kB]
Ge

In [5]:
!wget https://bootstrap.pypa.io/get-pip.py

--2024-04-14 19:07:20--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2635835 (2.5M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.51M  --.-KB/s    in 0.02s   

2024-04-14 19:07:20 (103 MB/s) - ‘get-pip.py’ saved [2635835/2635835]



In [6]:
!apt-get install python3.9-distutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3.9-lib2to3
The following NEW packages will be installed:
  python3.9-distutils python3.9-lib2to3
0 upgraded, 2 newly installed, 0 to remove and 46 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,234 kB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.9-lib2to3 all 3.9.19-1+jammy1 [127 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.9-distutils all 3.9.19-1+jammy1 [193 kB]
Fetched 319 kB in 0s (1,832 kB/s)
Selecting previously unselected package python3.9-lib2to3.
(Reading database ... 122417 files and directories currently installed.)
Preparing to unpack .../python3.9-lib2to3_3.9.19-1+jammy1_all.deb ...
Unpacking python3.9-lib2to3 (3.9.19-1+jammy1) ...
Selecting previously unselected package 

In [7]:
!python3.9 get-pip.py

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 11.2 MB/s eta 0:00:00
Using cached wheel-0.43.0-py3-none-any.whl (65 kB)


In [15]:
# remove jupyter from this in colab
!pip install -r requirements.txt

In [16]:
import lcdb
import json
import lcpfn
import torch as th
from functools import partial
import numpy as np
from matplotlib import pyplot as plt

DEFINE PATH

In [20]:
import os
PATH = os.path.join(os.getcwd(), 'notebooks')
PATH

'/content/deep-learning-project/notebooks'

In [21]:
# path = "C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\Year 1\\Q3\\Deep Learning\\deep-learning-project\\notebooks\data.json"

def readDatasetJson(path):
    f = open(path)

    dataset = json.load(f)
    # print(len(dataset))

    x = []
    y = []

    for v in dataset.values():
        x.append(v[0])
        y.append(v[1])

    x_true = th.Tensor(x)
    y_true = th.Tensor(y)

    return x_true, y_true

import os
def find_file(root_dir, target_filename):
    for root, dirs, files in os.walk(root_dir):
        if target_filename in files:
            return os.path.join(root, target_filename)

    # If the loop completes without returning, the file was not found
    return None

def find_data():
    return find_file(os.getcwd(), 'data.json')

def get_data():
    return readDatasetJson(find_data())

# function producing batches for PFN training
def get_batch(
    batch_size,
    seq_len,
    num_features,
    device="cpu",
    noisy_target=True,
    **_,
):
    assert num_features == 1

    x_data, y_data = get_data()
    print(x_data.shape)
    x_data = x_data[:batch_size, :]
    y_data = y_data[:batch_size, :]

    y_data_noisy = y_data.clone()

    x_data = x_data.view((num_features, batch_size, seq_len)).transpose(2, 0).to(device)
    y_data = y_data.transpose(1, 0).to(device)
    y_data_noisy = y_data.clone()

    return x_data, y_data, y_data_noisy

In [22]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch,
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1651,
                         epochs=15)

th.save(result[2].state_dict(), os.path.join(PATH, 'model_1651.txt'))

torch.Size([3302, 26])


RuntimeError: shape '[1, 10000, 26]' is invalid for input of size 85852

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch,
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1000,
                         epochs=15)

th.save(result[2].state_dict(), os.path.join(PATH, 'model_256_12.txt'))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch,
                         seq_len=26,
                         emsize=256,
                         nlayers=10,
                         lr=0.001,
                         batch_size=1000,
                         epochs=15)

th.save(result[2].state_dict(), os.path.join(PATH, 'model_256_10.txt'))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch,
                         seq_len=26,
                         emsize=256,
                         nlayers=18,
                         lr=0.001,
                         batch_size=1000,
                         epochs=15)

th.save(result[2].state_dict(), os.path.join(PATH, 'model_256_8.txt'))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch,
                         seq_len=26,
                         emsize=128,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1000,
                         epochs=15)

th.save(result[2].state_dict(), os.path.join(PATH, 'model_128_12.txt'))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch,
                         seq_len=26,
                         emsize=128,
                         nlayers=10,
                         lr=0.001,
                         batch_size=1000,
                         epochs=15)

th.save(result[2].state_dict(), os.path.join(PATH, 'model_128_10.txt'))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch,
                         seq_len=26,
                         emsize=128,
                         nlayers=8,
                         lr=0.001,
                         batch_size=1000,
                         epochs=15)

th.save(result[2].state_dict(), os.path.join(PATH, 'model_128_8.txt'))